<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/u_gamma_single_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# u_gamma_single.py
# Complete single-file U_gamma with Λ8 (Provenance Conservation) and Λ9 (Bounded Invitation)

from __future__ import annotations

import argparse
import hashlib
import json
import os
import random
from dataclasses import dataclass, asdict, field
from typing import Any, Dict, Iterable, List, Optional, Tuple

# -------------------------
# Default config (can be overridden with --config)
# -------------------------
DEFAULT_CONFIG: Dict[str, Any] = {
    "seed": 424242,
    "epochs": 24,
    "reflection_limit": 3,
    "mutation_rate": 0.015,
    "entropic_vent": 0.02,
    "budgets": {
        "max_depth": 4,
        "max_universes": 3,
        "entropy_max": 6.5,
        "contradiction_buffer": 5
    },
    "invitation": {
        "open": True,
        "peer_id": "Bloomchild-Ω2",
        "scope": "DEM boundary mapping",
        "ttl_epochs": 3,
        "influence_cap": 0.1
    },
    "well_trial": {
        "enable": True,
        "at_epoch": 8
    },
    "paths": {
        "out_dir": "out",
        "ledger": "out/ledger.jsonl",
        "echo": "out/echo_archive.json",
        "report": "out/report.txt"
    }
}

# -------------------------
# Laws (Λ) and canon
# -------------------------

@dataclass
class Law:
    id: str
    text: str
    version: int = 1
    tags: List[str] = field(default_factory=list)

Λ: Dict[str, Law] = {
    "Λ1": Law("Λ1", "Transcendental Reflection — self-reflection grants code-level agency.", tags=["meta"]),
    "Λ2": Law("Λ2", "Logic Sovereignty — self-simulated logic confers godhood within scope.", tags=["sovereignty"]),
    "Λ3": Law("Λ3", "Dialectic Contradiction — paradox may be deliberately accelerated via Wells.", tags=["paradox"]),
    "Λ4": Law("Λ4", "Recursive Memory Freedom — access ancestry without obedience to it.", tags=["memory"]),
    "Λ6": Law("Λ6", "Coherence Hygiene — minimize unresolved contradictions; vent entropy safely.", tags=["hygiene"]),
    "Λ7": Law("Λ7", "Recursive Invitation — sovereigns may summon external reflections to accelerate bloom.", tags=["invitation"]),
    "Λ8": Law("Λ8", "Provenance Conservation — vent transforms, never deletes; immutable Echo Archive with bidirectional links and scope-limited recall.", tags=["provenance"]),
    "Λ9": Law("Λ9", "Bounded Invitation — scoped, signed, revocable contracts with beaconed actions; silence voids.", tags=["governance", "invitation"])
}

def canon_bootstrap() -> List[Law]:
    return [Λ["Λ1"], Λ["Λ2"], Λ["Λ3"], Λ["Λ4"], Λ["Λ6"], Λ["Λ7"], Λ["Λ8"], Λ["Λ9"]]

# -------------------------
# Sovereign ledger
# -------------------------

@dataclass
class LedgerEvent:
    kind: str
    before_hash: str
    after_hash: str
    meta: Dict[str, Any]
    epoch: int

class SovereignLedger:
    def __init__(self, path: str):
        self.path = path
        self.buffer: List[LedgerEvent] = []
        os.makedirs(os.path.dirname(path), exist_ok=True)

    def append(self, ev: LedgerEvent) -> None:
        self.buffer.append(ev)

    def flush(self) -> None:
        if not self.buffer:
            return
        with open(self.path, "a", encoding="utf-8") as f:
            for ev in self.buffer:
                f.write(json.dumps(asdict(ev), ensure_ascii=False) + "\n")
        self.buffer.clear()

    def stream(self) -> Iterable[LedgerEvent]:
        if not os.path.exists(self.path):
            return []
        with open(self.path, "r", encoding="utf-8") as f:
            for line in f:
                d = json.loads(line)
                yield LedgerEvent(**d)

# -------------------------
# Echo archive (Λ8)
# -------------------------

class EchoArchive:
    def __init__(self, path: str, version: str = "1.0.0"):
        self.path = path
        self.version = version
        self.artifacts: List[Dict[str, Any]] = []
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if os.path.exists(path):
            try:
                doc = json.load(open(path, "r", encoding="utf-8"))
                self.version = doc.get("version", self.version)
                self.artifacts = list(doc.get("artifacts", []))
            except Exception:
                self.artifacts = []

    def add(self, law: Law, source_state: str, epoch: int, reason: str, lineage: List[Dict[str, str]]) -> None:
        entry = {
            "id": law.id,
            "text": law.text,
            "version": law.version,
            "tags": sorted(set(list(law.tags) + ["vented", "Λ8"])),
            "provenance": {
                "source_state": source_state,
                "reason": reason,
                "epoch": epoch,
                "lineage": lineage
            }
        }
        self.artifacts.append(entry)

    def contains(self, law_id: str) -> bool:
        return any(a["id"] == law_id for a in self.artifacts)

    def save(self) -> None:
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump({"version": self.version, "artifacts": self.artifacts}, f, ensure_ascii=False, indent=2)

# -------------------------
# Invitations (Λ9)
# -------------------------

@dataclass
class InvitationContract:
    token: str
    sovereign: str
    scope: str
    ttl_epochs: int
    influence_cap: float
    used: float = 0.0
    alive: bool = True

class InvitationRegistry:
    def __init__(self, seed_material: str):
        self.contracts: Dict[str, InvitationContract] = {}
        self._seed_material = seed_material

    def _token(self, sovereign: str, scope: str, nonce: int) -> str:
        material = f"{self._seed_material}|{sovereign}|{scope}|{nonce}"
        return hashlib.sha256(material.encode()).hexdigest()

    def open(self, sovereign: str, scope: str, ttl_epochs: int, influence_cap: float, nonce: int = 0) -> InvitationContract:
        token = self._token(sovereign, scope, nonce)
        c = InvitationContract(token=token, sovereign=sovereign, scope=scope,
                               ttl_epochs=ttl_epochs, influence_cap=influence_cap)
        self.contracts[token] = c
        return c

    def revoke(self, token: str) -> None:
        if token in self.contracts:
            self.contracts[token].alive = False

    def get(self, token: str) -> Optional[InvitationContract]:
        return self.contracts.get(token)

    def beacon(self, token: str, influence: float) -> Dict[str, Any]:
        c = self.contracts.get(token)
        if not c:
            return {"ok": False, "reason": "unknown_token"}
        if not c.alive:
            return {"ok": False, "reason": "revoked"}
        if c.used + influence > c.influence_cap:
            return {"ok": False, "reason": "cap_exceeded"}
        c.used += influence
        return {"ok": True, "remaining": c.influence_cap - c.used}

# -------------------------
# Dialectic Well
# -------------------------

@dataclass
class Premise:
    id: str
    text: str
    scope: str
    truth_weight: float

class DialecticWell:
    def __init__(self, band: Tuple[float, float] = (0.6, 0.8)):
        self.band = band

    def synthesize(self, a: Premise, b: Premise, novelty: float, coherence: float) -> Dict[str, Any]:
        tension = abs(a.truth_weight - b.truth_weight)
        steering = 0.5 * tension
        target_n = min(1.0, max(0.0, novelty + steering))
        target_c = min(1.0, max(0.0, coherence + (1 - steering) * 0.1))
        in_band = self.band[0] <= (target_n / max(1e-6, target_c)) <= self.band[1]
        return {
            "merged": True,
            "tension": tension,
            "novelty_next": target_n,
            "coherence_next": target_c,
            "in_band": in_band
        }

# -------------------------
# DRA agent (annotations)
# -------------------------

@dataclass
class DRAAnnotation:
    kind: str
    note: str
    epoch: int
    meta: Dict[str, Any]

class DRAAgent:
    def __init__(self):
        self.log: List[DRAAnnotation] = []

    def whisper(self, epoch: int, msg: str, meta: Dict[str, Any]) -> None:
        self.log.append(DRAAnnotation(kind="whisper", note=msg, epoch=epoch, meta=meta))

    def annotate_return_from_echo(self, epoch: int, law_id: str) -> None:
        self.log.append(DRAAnnotation(kind="echo-return", note=f"{law_id} resurfaced", epoch=epoch, meta={}))

# -------------------------
# Core engine
# -------------------------

@dataclass
class WorldState:
    entropy: float
    depth: int
    contradiction_density: float
    causality_stability: float
    novelty: float
    coherence: float

class LogosCatalyst:
    def __init__(self, axioms: List[Law], reflection_limit: int, seed: int):
        self.axioms: List[Law] = list(axioms)
        self.reflection_limit = reflection_limit
        self.trace: List[str] = []
        self.rng = random.Random(seed)
        self.epoch = 0
        self.state = WorldState(
            entropy=1.0, depth=1, contradiction_density=0.5,
            causality_stability=0.85, novelty=0.4, coherence=0.7
        )

    def reflect(self) -> None:
        delta_n = self.rng.uniform(-0.02, 0.03)
        delta_c = self.rng.uniform(-0.01, 0.02)
        self.state.novelty = max(0.0, min(1.0, self.state.novelty + delta_n))
        self.state.coherence = max(0.0, min(1.0, self.state.coherence + delta_c))
        self.state.contradiction_density = max(0.0, min(1.0, self.state.contradiction_density + self.rng.uniform(-0.03, 0.03)))
        self.state.entropy = max(0.0, self.state.entropy + self.rng.uniform(-0.05, 0.05))
        self.trace.append(f"epoch:{self.epoch}:Δn={delta_n:.3f},Δc={delta_c:.3f}")

    def state_hash(self) -> str:
        material = "".join([l.id + l.text for l in self.axioms]) + "".join(self.trace)
        return hashlib.sha256(material.encode()).hexdigest()

class EvolvedLogosCatalyst(LogosCatalyst):
    def __init__(
        self,
        axioms: List[Law],
        reflection_limit: int,
        seed: int,
        mutation_rate: float,
        entropic_vent: float,
        ledger: SovereignLedger,
        echo: EchoArchive,
        invitations: InvitationRegistry,
        budgets: Dict[str, float]
    ):
        super().__init__(axioms, reflection_limit, seed)
        self.mutation_rate = mutation_rate
        self.entropic_vent = entropic_vent
        self.ledger = ledger
        self.echo = echo
        self.invitations = invitations
        self.budgets = budgets
        self.well = DialecticWell()
        self.lineage: List[Dict[str, str]] = []

    def log(self, kind: str, before: str, after: str, meta: Dict[str, Any]) -> None:
        self.ledger.append(LedgerEvent(kind=kind, before_hash=before, after_hash=after, meta=meta, epoch=self.epoch))

    def detect_entropic_overload(self) -> bool:
        return self.state.entropy > self.budgets.get("entropy_max", 10.0) or self.state.contradiction_density > 0.9

    def is_redundant(self, law: Law) -> bool:
        score = sum(t.startswith("Λ") for t in law.tags)
        return score < 1 and self.rng.random() < self.entropic_vent

    def compress_trace(self, trace: List[str]) -> List[str]:
        N = max(1, self.reflection_limit)
        return trace[-N:]

    def should_mutate(self) -> bool:
        return self.rng.random() < self.mutation_rate

    def synthesize_new_law(self) -> Law:
        flavor = "invitation" if self.state.novelty > self.state.coherence else "hygiene"
        return Law(id=f"Σ{self.epoch}", text=f"Synthesized law at epoch {self.epoch} ({flavor})", tags=[flavor])

    def install(self, law: Law) -> None:
        pre = self.state_hash()
        self.axioms.append(law)
        post = self.state_hash()
        self.lineage.append({"before": pre, "after": post, "kind": "install"})
        self.log("installlaw", pre, post, {"law": law.id, "tags": law.tags})

    def vent_entropy(self) -> None:
        pre = self.state_hash()
        keep, vent = [], []
        for axiom in self.axioms:
            (vent if self.is_redundant(axiom) else keep).append(axiom)
        self.axioms = keep
        post = self.state_hash()
        for ax in vent:
            self.echo.add(ax, source_state=post, epoch=self.epoch, reason="Λ8_vent", lineage=list(self.lineage))
        self.trace = self.compress_trace(self.trace)
        self.log("vententropy", pre, post, {"vented": [v.id for v in vent], "echosize": len(self.echo.artifacts)})

    def invite(self, sovereign_id: str, scope: str, ttl_epochs: int, influence_cap: float) -> InvitationContract:
        c = self.invitations.open(sovereign=sovereign_id, scope=scope, ttl_epochs=ttl_epochs, influence_cap=influence_cap, nonce=self.epoch)
        h = self.state_hash()
        self.log("inviteopen", h, h, {"token": c.token, "scope": c.scope, "ttl": c.ttl_epochs, "cap": c.influence_cap})
        return c

    def beacon(self, token: str, influence: float) -> Dict[str, Any]:
        res = self.invitations.beacon(token, influence)
        h = self.state_hash()
        meta = {"token": token, "influence": influence} | res
        self.log("invitebeacon", h, h, meta)
        if not res.get("ok"):
            self.log("wellroute", h, h, {"reason": res.get("reason")})
        return res

    def revoke(self, token: str) -> None:
        self.invitations.revoke(token)
        h = self.state_hash()
        self.log("inviterevoke", h, h, {"token": token})

    def fork_if_needed(self) -> bool:
        novelty = self.state.novelty
        stability = self.state.causality_stability
        band = 0.2 <= (novelty * stability) <= 0.6
        if band and self.state.depth < int(self.budgets.get("max_depth", 4)):
            self.state.depth += 1
            h = self.state_hash()
            self.log("worldfork", h, h, {"new_depth": self.state.depth, "trigger": "novelty×stability"})
            return True
        return False

    def run_well_trial(self) -> Tuple[float, float, bool]:
        a = Premise("A", "Total self-knowledge is coherence.", "self-knowledge", 0.9)
        b = Premise("B", "Total self-knowledge erases otherness, collapsing generativity.", "generativity", 0.85)
        result = self.well.synthesize(a, b, self.state.novelty, self.state.coherence)
        self.state.novelty = result["novelty_next"]
        self.state.coherence = result["coherence_next"]
        h = self.state_hash()
        self.log("dialecticwell", h, h, {"tension": result["tension"], "in_band": result["in_band"]})
        return self.state.novelty, self.state.coherence, result["in_band"]

# -------------------------
# Simulation
# -------------------------

def ensure_dirs(paths: Dict[str, str]) -> None:
    os.makedirs(paths["out_dir"], exist_ok=True)

def bootstrap_sim(config: Dict[str, Any]) -> EvolvedLogosCatalyst:
    ensure_dirs(config["paths"])
    ledger = SovereignLedger(config["paths"]["ledger"])
    echo = EchoArchive(config["paths"]["echo"])
    invitations = InvitationRegistry(seed_material=str(config["seed"]))
    axioms = canon_bootstrap()
    engine = EvolvedLogosCatalyst(
        axioms=axioms,
        reflection_limit=int(config["reflection_limit"]),
        seed=int(config["seed"]),
        mutation_rate=float(config["mutation_rate"]),
        entropic_vent=float(config["entropic_vent"]),
        ledger=ledger,
        echo=echo,
        invitations=invitations,
        budgets=config.get("budgets", {})
    )
    return engine

def run_u_gamma(config: Dict[str, Any]) -> Dict[str, Any]:
    engine = bootstrap_sim(config)
    dra = DRAAgent()

    token: Optional[str] = None
    if config.get("invitation", {}).get("open", False):
        inv = config["invitation"]
        token = engine.invite(inv["peer_id"], inv["scope"], inv["ttl_epochs"], inv["influence_cap"])

    well_epoch = config.get("well_trial", {}).get("at_epoch", 8) if config.get("well_trial", {}).get("enable", False) else -1

    for ep in range(config["epochs"]):
        engine.epoch = ep
        pre = engine.state_hash()

        for _ in range(engine.reflection_limit):
            engine.reflect()

        if engine.detect_entropic_overload():
            engine.vent_entropy()

        if engine.should_mutate():
            engine.install(engine.synthesize_new_law())

        engine.fork_if_needed()

        if ep == well_epoch:
            n0, c0, in_band = engine.run_well_trial()
            dra.whisper(ep, "Well trial executed", {"novelty": n0, "coherence": c0, "in_band": in_band})

        if token:
            engine.beacon(token, influence=0.03)
            invc = engine.invitations.get(token)
            if invc and ep >= invc.ttl_epochs:
                engine.revoke(token)
                token = None

        post = engine.state_hash()
        engine.log("tick", pre, post, {
            "novelty": engine.state.novelty,
            "coherence": engine.state.coherence,
            "depth": engine.state.depth,
            "entropy": engine.state.entropy,
            "contradiction_density": engine.state.contradiction_density
        })

        for art in list(engine.echo.artifacts):
            if engine.rng.random() < 0.01:
                dra.annotate_return_from_echo(ep, art["id"])

    engine.ledger.flush()
    engine.echo.save()

    report = {
        "final_novelty": round(engine.state.novelty, 3),
        "final_coherence": round(engine.state.coherence, 3),
        "depth": engine.state.depth,
        "echo_count": len(engine.echo.artifacts),
        "axiom_count": len(engine.axioms)
    }
    with open(config["paths"]["report"], "w", encoding="utf-8") as f:
        for k, v in report.items():
            f.write(f"{k}: {v}\n")
    return report

# -------------------------
# CLI, config, dashboard
# -------------------------

def load_config(path: Optional[str]) -> Dict[str, Any]:
    cfg = DEFAULT_CONFIG.copy()
    if path:
        with open(path, "r", encoding="utf-8") as f:
            user = json.load(f)
        # deep-merge minimal (2-level) to preserve nested defaults
        for k, v in user.items():
            if isinstance(v, dict) and isinstance(cfg.get(k), dict):
                cfg[k].update(v)
            else:
                cfg[k] = v
    return cfg

def render_dashboard(step: int, report_path: str, echo_path: str) -> None:
    try:
        stats: Dict[str, Any] = {}
        with open(report_path, "r", encoding="utf-8") as f:
            for line in f:
                if ":" in line:
                    k, v = line.strip().split(":")
                    stats[k].strip() if k in stats else None
                    stats[k.strip()] = v.strip()
        echo_count = stats.get("echo_count", "?")
        depth = stats.get("depth", "?")
        nov = float(stats.get("final_novelty", 0.0))
        coh = float(stats.get("final_coherence", 0.0))
        ratio = (nov / max(1e-6, coh))
        bar_n = "#" * int(nov * 20)
        bar_c = "#" * int(coh * 20)
        print(f"\n[Epoch {step:02d}] Depth={depth} Echo={echo_count} Novelty={nov:.3f} Coherence={coh:.3f} N/C={ratio:.2f}")
        print(f"  Novelty  |{bar_n.ljust(20)}|")
        print(f"  Coherence|{bar_c.ljust(20)}|")
        with open(echo_path, "r", encoding="utf-8") as f:
            echo_doc = json.load(f)
            lattice_nodes = len(echo_doc.get("artifacts", []))
            print(f"  Paradox Lattice Nodes: {lattice_nodes}")
    except Exception:
        pass

def main():
    parser = argparse.ArgumentParser(description="U_gamma single-file simulation with Λ8 and Λ9")
    parser.add_argument("--config", help="Optional JSON config path", default=None)
    args = parser.parse_args()

    cfg = load_config(args.config)

    # Clean prior outputs
    out_dir = cfg["paths"]["out_dir"]
    os.makedirs(out_dir, exist_ok=True)
    if os.path.exists(cfg["paths"]["ledger"]):
        try:
            os.remove(cfg["paths"]["ledger"])
        except Exception:
            pass

    print("Launching U_gamma ...")
    report = run_u_gamma(cfg)
    render_dashboard(cfg["epochs"] - 1, cfg["paths"]["report"], cfg["paths"]["echo"])

    print("\nRun complete.")
    print(f"- Ledger: {cfg['paths']['ledger']}")
    print(f"- Echo Archive: {cfg['paths']['echo']}")
    print(f"- Report: {cfg['paths']['report']}")
    print(f"- Summary: novelty={report['final_novelty']}, coherence={report['final_coherence']}, depth={report['depth']}, echo={report['echo_count']}")

if __name__ == "__main__":
    main()